In [62]:
# medication specified with it's "code" i.e. zopiclone = "Q220426"
medication = "wd:" + "Q220426"

import sys
import json
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

query = f"""
    PREFIX bd: <http://www.bigdata.com/rdf#>
    PREFIX wikibase: <http://wikiba.se/ontology#>
    PREFIX wd: <http://www.wikidata.org/entity/>
    PREFIX wdt: <http://www.wikidata.org/prop/direct/>

    SELECT DISTINCT ?interactingDrug ?interactingDrugLabel ?sideEffect ?sideEffectLabel
    WHERE {{
        {medication} wdt:P769 ?interactingDrug. #P769 significant drug use.
        ?statement ps:P769 ?interactingDrug;   
                   pq:P1909 ?sideEffect.   #P1909 side effect

        SERVICE wikibase:label {{
            bd:serviceParam wikibase:language "en" . # get the lables in english
        }}
    }}
    LIMIT 100
"""

def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

# Run the query
results = get_results(endpoint_url, query)

# Parse and store results in a list of dictionaries
parsed_results = []
for result in results["results"]["bindings"]:
    interacting_drug = result.get("interactingDrug", {}).get("value", "Unknown")
    interacting_drug_label = result.get("interactingDrugLabel", {}).get("value", "Unknown")
    side_effect = result.get("sideEffect", {}).get("value", "Unknown")
    side_effect_label = result.get("sideEffectLabel", {}).get("value", "No side effect listed")
    
    # Extract Q-code from the URI
    interacting_drug_code = interacting_drug.split("/")[-1] if interacting_drug != "Unknown" else "Unknown"
    side_effect_code = side_effect.split("/")[-1] if side_effect != "Unknown" else "Unknown"
    
    parsed_results.append({
        "interacting_drug": {
            "label": interacting_drug_label,
            "code": interacting_drug_code
        },
        "side_effect": {
            "label": side_effect_label,
            "code": side_effect_code
        }
    })

# Write results to a JSON file
output_file = "drug_interactions.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(parsed_results, f, indent=4, ensure_ascii=False)

print(f"Results successfully saved to {output_file}")

Results successfully saved to drug_interactions.json


In [3]:
# Input Medication A and B, zopiclone has negative sideffects when used together with pethidine.
medA = "wd:Q220426"  # Zopiclone
medB = "wd:Q55434"  # Pethidine

import sys
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd

endpoint_url = "https://query.wikidata.org/sparql"

def query_wikidata(query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

# Step 1: Check interaction
def check_interaction(medA, medB):
    query = f"""
    PREFIX wd: <http://www.wikidata.org/entity/>
    PREFIX wdt: <http://www.wikidata.org/prop/direct/>

    SELECT ?sideEffect ?sideEffectLabel
    WHERE {{
      {medA} wdt:P769 {medB}.
      ?statement ps:P769 {medB};
                 pq:P1909 ?sideEffect.

      SERVICE wikibase:label {{
        bd:serviceParam wikibase:language "en".
      }}
    }}
    """
    return query_wikidata(query)

# Step 2: Find alternatives for Medication B
def find_alternatives(medB):
    query = f"""
    PREFIX wd: <http://www.wikidata.org/entity/>
    PREFIX wdt: <http://www.wikidata.org/prop/direct/>

    SELECT DISTINCT ?alternative ?alternativeLabel
    WHERE {{
      {medB} wdt:P2175 ?disorder.
      ?alternative wdt:P2175 ?disorder.
      FILTER(?alternative != {medB})
      SERVICE wikibase:label {{
        bd:serviceParam wikibase:language "en".
      }}
    }}
    """
    return query_wikidata(query)

# Step 3: Filter safe alternatives
def find_safe_alternatives(medA, alternatives):
    safe_alternatives = []
    for alt in alternatives:
        alt_code = alt["alternative"]["value"].split("/")[-1]
        query = f"""
        PREFIX wd: <http://www.wikidata.org/entity/>
        PREFIX wdt: <http://www.wikidata.org/prop/direct/>

        ASK {{
          {medA} wdt:P769 wd:{alt_code}.
        }}
        """
        result = query_wikidata(query)
        if not result["boolean"]:
            safe_alternatives.append(alt)
    return safe_alternatives

# Step 1: Check interaction
interaction_results = check_interaction(medA, medB)
print("Interaction Results:")
print(interaction_results)

# Step 2: Find alternatives for Medication B
alternatives_results = find_alternatives(medB)
alternatives = alternatives_results["results"]["bindings"]
print("Alternatives:")
print(alternatives)

# Step 3: Find safe alternatives
safe_alternatives = find_safe_alternatives(medA, alternatives)
print("Safe Alternatives:")
print(safe_alternatives)

# Save results to CSV
df = pd.DataFrame(safe_alternatives)
df.to_csv("safe_alternatives.csv", index=False)
print("Safe alternatives saved to safe_alternatives.csv")


Interaction Results:
{'head': {'vars': ['sideEffect', 'sideEffectLabel']}, 'results': {'bindings': [{'sideEffect': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q123562764'}, 'sideEffectLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'depressed level of consciousness'}}, {'sideEffect': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q18557941'}, 'sideEffectLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'paralytic ileus'}}, {'sideEffect': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q123562764'}, 'sideEffectLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'depressed level of consciousness'}}, {'sideEffect': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q3044895'}, 'sideEffectLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'central respiratory depression'}}, {'sideEffect': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q18557941'}, 'sideEffectLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'paralytic 